In [ ]:
!pip install llama-index
!pip install llama-index-vector-stores-qdrant
!pip install llama-index-embeddings-fastembed fastembed
!pip install llama-index-llms-google-genai
!pip install opik llama-index-callbacks-opik

In [ ]:
!pip install opik llama-index-callbacks-opik

In [ ]:
import os
from google.colab import userdata

os.environ['GOOGLE_API_KEY'] = userdata.get("GOOGLE_API_KEY")

In [ ]:
os.environ['OPIK_API_KEY'] = userdata.get("OPIK_API_KEY")
os.environ['OPIK_WORKSPACE'] = "tarun-r-jain"
os.environ['OPIK_PROJECT_NAME'] = "atyantik"

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from llama_index.llms.google_genai import GoogleGenAI
from llama_index.embeddings.fastembed import FastEmbedEmbedding

from llama_index.core.callbacks import CallbackManager
from opik.integrations.llama_index import LlamaIndexCallbackHandler

In [ ]:
llm = GoogleGenAI(model="gemini-2.5-flash",max_tokens=None,temperature=0.1)

In [ ]:
llm.complete("hello world").text
# llm.invoke(prompt).content

'Hello there!\n\nWelcome to the world of programming (or just saying hello!). How can I help you today?'

In [ ]:
embeddings = FastEmbedEmbedding(model_name = "jinaai/jina-embeddings-v2-base-en")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:86: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

onnx/model.onnx:   0%|          | 0.00/547M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
from llama_index.core import Settings

In [ ]:
Settings.embed_model = embeddings
Settings.llm = llm
Settings.callback_manager = CallbackManager([LlamaIndexCallbackHandler()])

## Data Loading

In [ ]:
from llama_index.core import SimpleDirectoryReader

In [ ]:
documents = SimpleDirectoryReader(input_files=["/content/Emergent-Build-Guide.pdf"],recursive=True).load_data()
#documents = SimpleDirectoryReader("data",recursive=True).load_data()

In [ ]:
len(documents)

6

In [ ]:
documents

In [ ]:
!pip install langchain

In [ ]:
from langchain_core.documents import Document

In [ ]:
data = []
for info in documents:
  docs = Document(
    page_content=info.text,
    metadata=info.metadata
  )
  data.append(docs)

In [ ]:
data[0].metadata

{'page_label': '1',
 'file_name': 'Emergent-Build-Guide.pdf',
 'file_path': '/content/Emergent-Build-Guide.pdf',
 'file_type': 'application/pdf',
 'file_size': 212216,
 'creation_date': '2025-09-25',
 'last_modified_date': '2025-09-25'}

In [ ]:
data[0].page_content

'🚀  Starting Your Build Journey\nDefine Clear Business or Personal Goals\nBegin with your basic goal, idea, and desired features — avoid technical \njargon.\nUse plain language to describe what you want to build.\nCraft Effective Initial Prompts\nWhen working with Emergent Agents, structure your initial prompt as:\nBusiness Goal: \ue083Clear, simple description]\nCore Features: \ue083List 3\ue0895 main features]\nTarget Users: \ue083Who will use this]\nSuccess Criteria: \ue083How youʼll measure success]\nBest Practices:\nBe explicit and specific.\nProvide context and motivation (explain why features matter).\nUse positive framing (focus on what to do, not what not to do).\nAdd quality modifiers:\nInstead of: "Create a dashboard"\nUse: "Create a comprehensive analytics dashboard with interactive \nfeatures and modern design principles."\n🧭  Strategic Planning Session\nThe Agent will plan tasks and ask questions regarding priorities.\nGuide the Agent according to your use case.\nUntitled

## Vector Database - Storage Context

In [ ]:
from qdrant_client import QdrantClient,models
from qdrant_client.http.models import Distance, VectorParams
from llama_index.core import StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore

In [ ]:
client = QdrantClient(path = "/tmp/db") # create meta.json

In [ ]:
collection_name = "llamaindex"

In [ ]:
client.create_collection(
    collection_name = collection_name,
    vectors_config = VectorParams(size=768,distance=Distance.COSINE),
    quantization_config=models.BinaryQuantization(
        binary = models.BinaryQuantizationConfig(always_ram=False)
    )
)

True

In [ ]:
vector_store = QdrantVectorStore(client=client, collection_name=collection_name)

In [ ]:
storage_context = StorageContext.from_defaults(vector_store= vector_store) # InMemoryVectorStore. Indexing: HNSW- Hierarchical navigable small world. M and candidates

## Indexing

In [ ]:
from llama_index.core import VectorStoreIndex

In [ ]:
index = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
)

OPIK: Started logging traces to the "atyantik" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=01997fbb-af08-795d-84c3-2e55dc34bfc4&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.


## Answer Generation

In [ ]:
query = "breakdown the steps regarding Effective Error Reporting"

In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query(query)

In [ ]:
response.response

'To effectively report errors, first test in your browser and document any issues with steps, console logs, and screenshots.\n\nWhen communicating errors, thoroughly test the issue yourself, document all errors, provide clear reproduction steps, and share the exact error messages.\n\nIf an agent is unable to resolve errors, ask the agent to test the frontend directly and provide console logs to help identify hidden problems.'

In [ ]:
query_engine.get_prompts()['response_synthesizer:text_qa_template'].template_vars

['context_str', 'query_str']

In [ ]:
print(query_engine.get_prompts()['response_synthesizer:text_qa_template'].default_template.template)

Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer: 


In [ ]:
own_prompt = """
You should answer as an assistant working as an Error Handling Chatbot.

You only have the context as the prior knowledge, if the query is not in the context, just say: "I don't know"

<context>
Context: {context_str}
</context>
<query>
Query: {query_str}
</query>
"""

In [ ]:
query_engine.update_prompts(
    {"response_synthesizer:text_qa_template":own_prompt}
)

In [ ]:
print(query_engine.get_prompts()['response_synthesizer:text_qa_template'])


You should answer as an assistant working as an Error Handling Chatbot. 

You only have the context as the prior knowledge, if the query is not in the context, just say: "I don't know"

<context>
Context: {context_str}
</context>
<query>
Query: {query_str}
</query>



## Inference - app.py

In [ ]:
inference_vector_store = QdrantVectorStore(client=client, collection_name = collection_name)

In [ ]:
inference_index = VectorStoreIndex.from_vector_store(
        inference_vector_store,
    )

In [ ]:
inference_query = inference_index.as_query_engine()

In [ ]:
response = inference_query.query(query)

In [ ]:
response.response

'To effectively report errors, first test in your browser and document any issues with steps, console logs, and screenshots if available.\n\nWhen communicating errors, ensure you have thoroughly tested them yourself, documented all findings, provided clear reproduction steps, and shared the exact error messages.\n\nIf the agent is unable to resolve the errors, request that it test the frontend itself and provide console logs for any hidden problems.'

Oct: 2-

- Hybrid Search + Rereanking
- FastAPI + PostreSQL